In [1]:
import json
from collections import OrderedDict

# Load the original JSON data from a file
with open('/hpc2hdd/home/cguo847/Qwen-VL/finetune/grounding_dino+sam.json', 'r') as f:
    data = json.load(f)

# Create a new list to hold the modified data
new_data = []

# Process each entry in the data
for entry in data:
    # Create an ordered dictionary for the new entry
    new_entry = OrderedDict()
    
    # Add the 'id' key first
    new_entry['id'] = entry.pop('unique_id')
    
    # Remove unwanted keys
    del entry['image_id']
    del entry['data_source']
    
    # Modify the image path
    entry["image"] = f"/hpc2hdd/home/cguo847/train2017/{entry['image']}"
    
    # Iterate over conversations and modify specific messages
    for i, conversation in enumerate(entry['conversations']):
        if conversation['from'] == 'gpt':
            conversation['from'] = 'assistant'
            if 'actions' in conversation and conversation['actions']:
                conversation['value'] = f"<gsam>{conversation['actions'][0]['API_params']['caption']}</gsam>"
                del conversation['thoughts']
                del conversation['actions']
            elif 'thoughts' in conversation:
                del conversation['thoughts']
                del conversation['actions']
        elif conversation['from'] == 'human' and '\n<image>' in conversation['value']:
            # Swap the position of '<image>' and '\n'
            parts = conversation['value'].split('\n', 1)
            new_value = f"Picture 1: <img>{entry['image']}</img>\n{parts[0]}"
            conversation['value'] = new_value
        elif conversation['from'] == 'human' and i == 0:
            parts = conversation['value'].split('\n', 1)
            new_value = f"Picture 1: <img>{entry['image']}</img>\n{parts[1]}"
            conversation['value'] = new_value

        if conversation['from'] == 'human':
            conversation['from'] = 'user'


    # Add the 'conversations' key after 'id'
    new_entry['conversations'] = entry['conversations']
    
    # Add the 'image' key last
    new_entry['image'] = entry['image']

    # Remove the 'image' key from new_entry
    del new_entry['image']
    
    # Append the new entry to the new data list
    new_data.append(new_entry)

# Save the modified data to a new JSON file
with open('converted_data.json', 'w', encoding='utf-8') as f:
    json.dump(new_data, f, indent=2, ensure_ascii=False)

print("Data modification complete.")

Data modification complete.
